<a href="https://colab.research.google.com/github/nandini-mazumdar/learning-spark-again/blob/main/Intro_to_Spark_Chap_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

#Loading Data

##CSV Files

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving location_temp.csv to location_temp.csv
Saving server_name.csv to server_name.csv
Saving utilization.csv to utilization.csv


In [ ]:
df = spark.read.csv('location_temp.csv',inferSchema=True, header=True)
df.show(5, False)

+-------------------+-----------+------------+
|event_date         |location_id|temp_celcius|
+-------------------+-----------+------------+
|03/04/2019 19:48:06|loc0       |29          |
|03/04/2019 19:53:06|loc0       |27          |
|03/04/2019 19:58:06|loc0       |28          |
|03/04/2019 20:03:06|loc0       |30          |
|03/04/2019 20:08:06|loc0       |27          |
+-------------------+-----------+------------+
only showing top 5 rows



In [ ]:
df.take(10)

[Row(event_date='03/04/2019 19:48:06', location_id='loc0', temp_celcius=29),
 Row(event_date='03/04/2019 19:53:06', location_id='loc0', temp_celcius=27),
 Row(event_date='03/04/2019 19:58:06', location_id='loc0', temp_celcius=28),
 Row(event_date='03/04/2019 20:03:06', location_id='loc0', temp_celcius=30),
 Row(event_date='03/04/2019 20:08:06', location_id='loc0', temp_celcius=27),
 Row(event_date='03/04/2019 20:13:06', location_id='loc0', temp_celcius=27),
 Row(event_date='03/04/2019 20:18:06', location_id='loc0', temp_celcius=27),
 Row(event_date='03/04/2019 20:23:06', location_id='loc0', temp_celcius=29),
 Row(event_date='03/04/2019 20:28:06', location_id='loc0', temp_celcius=32),
 Row(event_date='03/04/2019 20:33:06', location_id='loc0', temp_celcius=35)]

In [ ]:
df.count()

500000

In [ ]:
df2 = spark.read.csv('utilization.csv',inferSchema=True, header=False)
df2.show(5, False)

+-------------------+---+----+----+---+
|_c0                |_c1|_c2 |_c3 |_c4|
+-------------------+---+----+----+---+
|03/05/2019 08:06:14|100|0.57|0.51|47 |
|03/05/2019 08:11:14|100|0.47|0.62|43 |
|03/05/2019 08:16:14|100|0.56|0.57|62 |
|03/05/2019 08:21:14|100|0.57|0.56|50 |
|03/05/2019 08:26:14|100|0.35|0.46|43 |
+-------------------+---+----+----+---+
only showing top 5 rows



In [ ]:
df2.count()

500000

In [ ]:
df2.take(10)

[Row(_c0='03/05/2019 08:06:14', _c1=100, _c2=0.57, _c3=0.51, _c4=47),
 Row(_c0='03/05/2019 08:11:14', _c1=100, _c2=0.47, _c3=0.62, _c4=43),
 Row(_c0='03/05/2019 08:16:14', _c1=100, _c2=0.56, _c3=0.57, _c4=62),
 Row(_c0='03/05/2019 08:21:14', _c1=100, _c2=0.57, _c3=0.56, _c4=50),
 Row(_c0='03/05/2019 08:26:14', _c1=100, _c2=0.35, _c3=0.46, _c4=43),
 Row(_c0='03/05/2019 08:31:14', _c1=100, _c2=0.41, _c3=0.58, _c4=48),
 Row(_c0='03/05/2019 08:36:14', _c1=100, _c2=0.57, _c3=0.35, _c4=58),
 Row(_c0='03/05/2019 08:41:14', _c1=100, _c2=0.41, _c3=0.4, _c4=58),
 Row(_c0='03/05/2019 08:46:14', _c1=100, _c2=0.53, _c3=0.35, _c4=62),
 Row(_c0='03/05/2019 08:51:14', _c1=100, _c2=0.51, _c3=0.6, _c4=45)]

In [ ]:
df2 = df2.withColumnRenamed("_c0","Event_datetime") \
      .withColumnRenamed("_c1","server_id") \
      .withColumnRenamed("_c2","cpu_utilization") \
      .withColumnRenamed("_c3","free_memory") \
      .withColumnRenamed("_c4","session_count")

In [ ]:
df2.show(10)

+-------------------+---------+---------------+-----------+-------------+
|     Event_datetime|server_id|cpu_utilization|free_memory|session_count|
+-------------------+---------+---------------+-----------+-------------+
|03/05/2019 08:06:14|      100|           0.57|       0.51|           47|
|03/05/2019 08:11:14|      100|           0.47|       0.62|           43|
|03/05/2019 08:16:14|      100|           0.56|       0.57|           62|
|03/05/2019 08:21:14|      100|           0.57|       0.56|           50|
|03/05/2019 08:26:14|      100|           0.35|       0.46|           43|
|03/05/2019 08:31:14|      100|           0.41|       0.58|           48|
|03/05/2019 08:36:14|      100|           0.57|       0.35|           58|
|03/05/2019 08:41:14|      100|           0.41|        0.4|           58|
|03/05/2019 08:46:14|      100|           0.53|       0.35|           62|
|03/05/2019 08:51:14|      100|           0.51|        0.6|           45|
+-------------------+---------+-------

##JSON/Text Files

In [ ]:
from google.colab import files
files.upload()

Saving utilization.json to utilization.json


In [ ]:
json_df = spark.read.format('json').load('/utilization.json')

In [ ]:
json_df.show(10)

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
+---------------+-------------------+-

In [ ]:
json_df.count()

500000

In [ ]:
json_df.columns

['cpu_utilization',
 'event_datetime',
 'free_memory',
 'server_id',
 'session_count']

#Dataframe Operations

##Sampling

In [ ]:
df_sample = json_df.sample(withReplacement=False, fraction=0.1, seed=611)
df_sample.show()

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.66|03/05/2019 09:06:14|       0.72|      100|           57|
|           0.56|03/05/2019 10:26:14|       0.69|      100|           44|
|           0.32|03/05/2019 11:01:14|       0.56|      100|           53|
|            0.5|03/05/2019 12:16:14|        0.5|      100|           51|
|           0.55|03/05/2019 13:16:14|       0.69|      100|           66|
|           0.39|03/05/2019 15:01:14|       0.52|      100|           48|
|           0.56|03/05/2019 16:41:14|       0.59|      100|           65|
|           0.66|03/05/2019 16:51:14|       0.35|      100|           64|
|            0.4|03/05/2019 17:21:14|       0.41|      100|           43|
|            0.4|03/05/2019 17:51:14|       0.51|      100|           53|
|           0.28|03/05/2019 18:31:14| 

In [ ]:
df_sample.count()

50171

In [ ]:
df_sample.groupBy('server_id').agg({'cpu_utilization':'mean'}).orderBy('server_id').show(10)

+---------+--------------------+
|server_id|avg(cpu_utilization)|
+---------+--------------------+
|      100|  0.4574302788844616|
|      101|   0.800517412935325|
|      102|  0.7549898785425116|
|      103|  0.7627385496183201|
|      104|  0.7088264794383129|
|      105| 0.49335317460317507|
|      106| 0.42642192347466434|
|      107|  0.6552186588921286|
|      108|  0.7464645426515926|
|      109|  0.5614764397905764|
+---------+--------------------+
only showing top 10 rows



In [ ]:
json_df.groupBy('server_id').agg({'cpu_utilization':'mean'}).orderBy('server_id').show(10)

+---------+--------------------+
|server_id|avg(cpu_utilization)|
+---------+--------------------+
|      100|   0.467506000000003|
|      101|  0.7985559999999872|
|      102|  0.7583949999999904|
|      103|  0.7614389999999969|
|      104|  0.7108530000000015|
|      105| 0.49256400000000206|
|      106|  0.4220220000000024|
|      107|  0.6505060000000048|
|      108|  0.7476360000000036|
|      109|  0.5630629999999914|
+---------+--------------------+
only showing top 10 rows



##Sorting

In [ ]:
df_sort = df_sample.sort('event_datetime')
df_sort.show()

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.61|03/05/2019 08:06:17|       0.12|      102|           71|
|           0.64|03/05/2019 08:06:19|       0.32|      103|           96|
|           0.51|03/05/2019 08:06:46|       0.49|      119|           53|
|            0.5|03/05/2019 08:07:09|       0.49|      132|           65|
|           0.85|03/05/2019 08:07:41|       0.23|      148|           91|
|           0.58|03/05/2019 08:11:31|       0.61|      110|           76|
|           0.58|03/05/2019 08:11:48|       0.38|      120|           64|
|           0.33|03/05/2019 08:11:54|       0.75|      124|           41|
|           0.47|03/05/2019 08:12:00|       0.32|      127|           65|
|           0.46|03/05/2019 08:12:31|       0.56|      143|           52|
|           0.91|03/05/2019 08:16:17| 

##Filtering

In [ ]:
df_time = spark.read.csv('/location_temp.csv', header=True, inferSchema=True)
df_time.show(5)

+-------------------+-----------+------------+
|         event_date|location_id|temp_celcius|
+-------------------+-----------+------------+
|03/04/2019 19:48:06|       loc0|          29|
|03/04/2019 19:53:06|       loc0|          27|
|03/04/2019 19:58:06|       loc0|          28|
|03/04/2019 20:03:06|       loc0|          30|
|03/04/2019 20:08:06|       loc0|          27|
+-------------------+-----------+------------+
only showing top 5 rows



In [ ]:
df_time.filter(df_time['location_id']=="loc0").show(10)

+-------------------+-----------+------------+
|         event_date|location_id|temp_celcius|
+-------------------+-----------+------------+
|03/04/2019 19:48:06|       loc0|          29|
|03/04/2019 19:53:06|       loc0|          27|
|03/04/2019 19:58:06|       loc0|          28|
|03/04/2019 20:03:06|       loc0|          30|
|03/04/2019 20:08:06|       loc0|          27|
|03/04/2019 20:13:06|       loc0|          27|
|03/04/2019 20:18:06|       loc0|          27|
|03/04/2019 20:23:06|       loc0|          29|
|03/04/2019 20:28:06|       loc0|          32|
|03/04/2019 20:33:06|       loc0|          35|
+-------------------+-----------+------------+
only showing top 10 rows



In [ ]:
df_time.filter(df_time['location_id']=="loc0").count()

1000

In [ ]:
df_time.select('location_id').distinct().sort('location_id').show()

+-----------+
|location_id|
+-----------+
|       loc0|
|       loc1|
|      loc10|
|     loc100|
|     loc101|
|     loc102|
|     loc103|
|     loc104|
|     loc105|
|     loc106|
|     loc107|
|     loc108|
|     loc109|
|      loc11|
|     loc110|
|     loc111|
|     loc112|
|     loc113|
|     loc114|
|     loc115|
+-----------+
only showing top 20 rows



##Aggregation

In [ ]:
df_time.columns

['event_date', 'location_id', 'temp_celcius']

In [ ]:
df_time.groupBy('location_id').count().show()

+-----------+-----+
|location_id|count|
+-----------+-----+
|     loc196| 1000|
|     loc226| 1000|
|     loc463| 1000|
|     loc150| 1000|
|     loc292| 1000|
|     loc311| 1000|
|      loc22| 1000|
|     loc351| 1000|
|     loc370| 1000|
|     loc419| 1000|
|      loc31| 1000|
|     loc305| 1000|
|      loc82| 1000|
|      loc90| 1000|
|     loc118| 1000|
|     loc195| 1000|
|     loc208| 1000|
|      loc39| 1000|
|      loc75| 1000|
|     loc228| 1000|
+-----------+-----+
only showing top 20 rows



In [ ]:
df_time.groupBy('location_id').agg({'temp_celcius':'mean'}).show()

+-----------+-----------------+
|location_id|avg(temp_celcius)|
+-----------+-----------------+
|     loc196|           29.225|
|     loc226|           25.306|
|     loc463|           23.317|
|     loc150|           32.188|
|     loc292|           29.159|
|     loc311|           24.308|
|      loc22|           28.251|
|     loc351|           28.194|
|     loc370|            29.14|
|     loc419|           29.141|
|      loc31|           25.196|
|     loc305|           27.314|
|      loc82|           27.355|
|      loc90|           23.216|
|     loc118|           24.219|
|     loc195|            27.25|
|     loc208|           26.206|
|      loc39|           25.199|
|      loc75|           23.209|
|     loc228|           27.295|
+-----------+-----------------+
only showing top 20 rows



#Write Dataframe to file

In [ ]:
df_sample.write.csv('df_sample.csv')

In [ ]:
! ls df_sample.csv

part-00000-b27803b5-2d09-4b21-ab61-749dbb0d4f95-c000.csv  _SUCCESS
part-00001-b27803b5-2d09-4b21-ab61-749dbb0d4f95-c000.csv


In [ ]:
! head df_sample.csv/part-00000-b27803b5-2d09-4b21-ab61-749dbb0d4f95-c000.csv

0.66,03/05/2019 09:06:14,0.72,100,57
0.56,03/05/2019 10:26:14,0.69,100,44
0.32,03/05/2019 11:01:14,0.56,100,53
0.5,03/05/2019 12:16:14,0.5,100,51
0.55,03/05/2019 13:16:14,0.69,100,66
0.39,03/05/2019 15:01:14,0.52,100,48
0.56,03/05/2019 16:41:14,0.59,100,65
0.66,03/05/2019 16:51:14,0.35,100,64
0.4,03/05/2019 17:21:14,0.41,100,43
0.4,03/05/2019 17:51:14,0.51,100,53
